# Data Collection - Conceptual Captions
In this document, I'll download the images from Google's Conceptual Captions dataset. (https://ai.google.com/research/ConceptualCaptions/download)

In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv('../DATA/Train_GCC-training.tsv', delimiter='\t', header=None, names = ['description', 'url'])

In [4]:
val_df = pd.read_csv('../DATA/Validation_GCC-1.1.0-Validation.tsv', delimiter = '\t', header = None, names = ['description', 'url'])

### Download images
Now I'll make a call to individual url to download images to the local drive.

In [5]:
import requests
import sys
import time
from PIL import Image
import os

In [6]:
path = '../DATA/IMAGES/Conceptual_Captions'

In [7]:
flist = os.listdir(path)

In [8]:
# downloading images
def download_img(ind, url, path, flist = None, pause = False): 
    if flist == None: 
        flist = os.listdir(path)
    
    print(f"Processing {ind} ", end = '...')
    
    if f"{ind}.jpg" in flist: 
        print(f"{ind}.jpg exists.", end = '.')
        status = 'saved'
    
    if url.startswith('http'):
        
        try: 
            # save the file
                fpath = f"{path}/{ind}.jpg"
                
            if status != 'saved': 
                resp = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'})

                with open(fpath, 'wb') as fp:
                    fp.write(resp.content)

                status = 'saved'

            try:
                # resizing image
                img = Image.open(fpath)
                img.thumbnail((500, 500))
                img.save(fpath)
                status = 'complete'
                print('Complete')

            except: 
                print(f'Failed to resize. \n{sys.exc_info()[0]}')
                status = 'failed: resizing'

        except:
            print(f'Failed to save. \n{sys.exc_info()[0]}')
            status = 'failed: saving'


    else:
        status = f'failed: invalid_url {url[:15]}...'
    
    if pause: 
        pause = np.random.randint(1, 60)
        time.sleep(pause) # buffer
        
    return status

In [9]:
tmp = train_df.copy()

In [10]:
tmp['status'] = None

In [11]:
import joblib

In [ ]:
for ind, url in enumerate(tmp['url'][:30000]): 
    status = download_img(ind, url, path, flist = flist) 
    if status: 
        tmp['status'][ind] = status
    # save the log every 1000 runs
    if ind % 1000 == 0: 
        joblib.dump(tmp, '../TMP/tmp')
    